In [5]:
number = 1
current_depth = 0
depth = 3
# Want.  alist that looks like [1, [[2 , [[3, []], [3, []], [3, []] ]], [2, [...]], [2, [...]]]]
def recursive_files(number, depth, current_depth=0):
    level_list = []
    if current_depth < depth:
        next_depth = current_depth + 1
        for i in range(0,next_depth):
            level_list.append((i, recursive_files(number, depth, next_depth)))
    level_list.append(number)
    return(level_list)

recursed_list = recursive_files(number, depth)
print(recursed_list)
for i in recursed_list:
    print(i)

[(0, [(0, [(0, [1]), (1, [1]), (2, [1]), 1]), (1, [(0, [1]), (1, [1]), (2, [1]), 1]), 1]), 1]
(0, [(0, [(0, [1]), (1, [1]), (2, [1]), 1]), (1, [(0, [1]), (1, [1]), (2, [1]), 1]), 1])
1


In [9]:
import sys
sys.path.append('/Users/jreid/Library/CloudStorage/Dropbox/programming/Projects/common_place')
from backend.src.modules.utils.database_utils import get_db_connection

def add_clustering_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        # Create clusters table
        cursor.execute("""
            CREATE TABLE clusters (
                cluster_id SERIAL PRIMARY KEY,
                parent_cluster_id INTEGER REFERENCES clusters(cluster_id),
                title VARCHAR(255),
                summary TEXT,
                layer INTEGER,
                created_at TIMESTAMPTZ DEFAULT NOW(),
                filters_used JSONB
            );
        """)
        
        # Create cluster_points table
        cursor.execute("""
            CREATE TABLE cluster_points (
                cluster_id INTEGER REFERENCES clusters(cluster_id) ON DELETE CASCADE,
                point_id BIGINT REFERENCES point(point_id) ON DELETE CASCADE,
                PRIMARY KEY (cluster_id, point_id)
            );
        """)
        
        # Add indexes
        indexes = [
            "CREATE INDEX idx_clusters_parent ON clusters(parent_cluster_id);",
            "CREATE INDEX idx_clusters_layer ON clusters(layer);",
            "CREATE INDEX idx_cluster_points_cluster ON cluster_points(cluster_id);",
            "CREATE INDEX idx_cluster_points_point ON cluster_points(point_id);"
        ]
        
        for index_sql in indexes:
            cursor.execute(index_sql)
        
        conn.commit()
        print("✅ Clustering tables and indexes created successfully!")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error creating tables: {e}")
        
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    add_clustering_tables()

✅ Clustering tables and indexes created successfully!
